<a href="https://colab.research.google.com/github/keinpyisi/Database/blob/main/CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# importing modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model
import requests

In [34]:
#Getting Encoder for Myanmar Chars
converter = "https://raw.githubusercontent.com/keinpyisi/Database/main/chatbot/converter.json"
response = requests.get(converter)
open("converter.json", "wb").write(response.content)

4113

In [35]:
#Getting Data To Train
rawcsv = "https://raw.githubusercontent.com/keinpyisi/Database/main/chatbot/training_data_eng_tag.csv"
response = requests.get(rawcsv)
open("training_data_eng_tag.csv", "wb").write(response.content)

33561

In [36]:
def encode(character):
  cha=character
  
  # Opening JSON file
  f = open('converter.json')
    
  # returns JSON object as 
  # a dictionary
  data = json.load(f)

  # Iterating through the json
  # list
  for i in data:
      fromdata=i.get('from')
      if(fromdata.__eq__(character)):
        # Closing file
        f.close()
        return i.get('to')

    

In [37]:
def decode(character):
  cha=character
  
  # Opening JSON file
  f = open('converter.json')
    
  # returns JSON object as 
  # a dictionary
  data = json.load(f)

  # Iterating through the json
  # list
  for i in data:
      fromdata=i.get('to')
      if(fromdata.__eq__(character)):
        # Closing file
        f.close()
        return i.get('from')

    
  

In [ ]:
from csv import reader
import json 
import csv
# open file in read mode
with open('training_data_eng_tag.csv', 'r') as read_obj:
        # name of csv file 
    filename = "training_data.csv"
    csv_reader = reader(read_obj)
    
        
    # writing to csv file 
    with open(filename, 'w') as csvfile: 
      csv_output = csv.writer(csvfile)
      # pass the file object to reader() to get the reader object
      
      i=next(csv_reader, None)  # skip the headers
      csv_output.writerow(i)
      # Iterate over each row in the csv using reader object
      for row in csv_reader:
          # row variable is a list that represents a row in csv
          append=""
          for char in row[0]:
            if(encode(char) is None):
              print(char)
              break
            char=encode(char)
            append+=char
          
          row[0]=append
          csv_output.writerow(row)
          print(append)
        


In [39]:
# importing training data
training_data = pd.read_csv("training_data.csv")

In [40]:
# preprocessing training data
#training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
training_data_tfidf = vectorizer.fit_transform(training_data["patterns"]).toarray()

In [41]:
# preprocessing target variable(tags)
le = LabelEncoder()
training_data_tags_le = pd.DataFrame({"tags": le.fit_transform(training_data["tags"])})
training_data_tags_dummy_encoded = pd.get_dummies(training_data_tags_le["tags"]).to_numpy()

In [42]:
# creating DNN
chatbot = Sequential()
chatbot.add(Dense(10, input_shape=(len(training_data_tfidf[0]),)))
chatbot.add(Dense(8))
chatbot.add(Dense(8))
chatbot.add(Dense(6))
chatbot.add(Dense(6))


chatbot.add(Dense(len(training_data_tags_dummy_encoded[0]), activation="softmax"))
chatbot.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# fitting DNN
chatbot.fit(training_data_tfidf, training_data_tags_dummy_encoded, epochs=50, batch_size=32)

In [44]:
# saving model file
save_model(chatbot, "chatbot")

INFO:tensorflow:Assets written to: chatbot/assets


In [45]:
# importing modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.models import load_model
import json
import random

In [46]:
# importing training data
training_data = pd.read_csv("training_data.csv")
# loading model
chatbot = load_model("chatbot")
# loading responses
#responses = json.load(open("responses.json", "r"))

In [47]:
# fitting TfIdfVectorizer with training data to preprocess inputs
#training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorizer.fit(training_data["patterns"])

TfidfVectorizer(ngram_range=(1, 2))

In [48]:
# fitting LabelEncoder with target variable(tags) for inverse transformation of predictions
le = LabelEncoder()
le.fit(training_data["tags"])

LabelEncoder()

In [49]:
# transforming input and predicting intent
def predict_tag(inp_str):
    inp_data_tfidf = vectorizer.transform([inp_str]).toarray()
    predicted_proba = chatbot.predict(inp_data_tfidf)
    encoded_label = [np.argmax(predicted_proba)]
    predicted_tag = le.inverse_transform(encoded_label)[0]
    
    return predicted_tag

In [50]:
# defining chat function
def start_chat():
    print("---------------  AI Chat bot  ---------------")
    print("Ask any queries...")
    print("I will try to understand you and reply...")
    print("Type EXIT to quit...")
    while True:
        inp = input("Ask anything... : ")
        print(encode(inp))
        if inp == "EXIT":
            break
        else:
            append=""
            for char in inp:
              if(encode(char) is None):
                print(char)
                break
              char=encode(char)
              append+=char
            if inp:
                tag = predict_tag(append)
                #Return Tag
                #response = random.choice(responses[tag])
                #Search Response with Tag and Random it 
                print("Response... : ", tag)
            else:
                pass

In [51]:
# calling chat function to start chatting
start_chat()

---------------  AI Chat bot  ---------------
Ask any queries...
I will try to understand you and reply...
Type EXIT to quit...
Ask anything... : ပျင်းလိုက်တာ
None
Response... :  other admissions


KeyboardInterrupt: ignored